# Stage 06 — Data Preprocessing (CCR Project)

**Goals:** Handle missing data, drop invalid rows, scale features, correct types, and document assumptions for reproducibility.  
**Steps:** load raw → fill_missing_median → drop_missing → normalize_data → save cleaned → compare vs original.

**Key assumptions (from reading):**
- Median imputation assumes MCAR/MAR; FFILL/BFILL assumes temporal continuity.
- Removing invalid values assumes they are errors; thresholds must be justified.
- StandardScaler assumes ~normal features; MinMax is sensitive to outliers.
- Document choices for transparency and reproducibility. 

In [1]:
from datetime import datetime
from pathlib import Path
import pandas as pd
import numpy as np

try:
    from src.config import load_env, get_path
    load_env()
    RAW_DIR = get_path("DATA_DIR_RAW", "data/raw")
    PROC_DIR = get_path("DATA_DIR_PROCESSED", "data/processed")
except Exception:
    RAW_DIR = Path("../data/raw")
    PROC_DIR = Path("../data/processed")

RAW_DIR, PROC_DIR


(WindowsPath('../data/raw'), WindowsPath('../data/processed'))

In [3]:
raw_path = RAW_DIR / "api_yfinance_JPM_20250821-0353.csv"   
df_raw = pd.read_csv(raw_path)

for cand in ["date", "Date", "timestamp", "Datetime"]:
    if cand in df_raw.columns:
        df_raw[cand] = pd.to_datetime(df_raw[cand], errors="coerce")

df_raw.info(), df_raw.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    125 non-null    datetime64[ns]
 1   Close   126 non-null    object        
 2   High    126 non-null    object        
 3   Low     126 non-null    object        
 4   Open    126 non-null    object        
 5   Volume  126 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 6.0+ KB


(None,
         Date               Close                High                 Low  \
 0        NaT                 JPM                 JPM                 JPM   
 1 2025-02-21  261.36322021484375  266.26924263014877   259.9784679949329   
 2 2025-02-24   258.4947814941406   264.0931645598592  255.79449220326956   
 3 2025-02-25  254.59767150878906  260.13671015757575  250.59177606908855   
 4 2025-02-26  255.97256469726562   258.0694794519864  254.13279879950727   
 
                  Open    Volume  
 0                 JPM       JPM  
 1  265.39881832468063  12835600  
 2  262.59959430697455  10372800  
 3   259.3751040401838   9608400  
 4  254.36030565887174   5943600  )

In [4]:
missing_counts = df_raw.isna().sum().sort_values(ascending=False)
missing_counts.head(10)


Date      1
Close     0
High      0
Low       0
Open      0
Volume    0
dtype: int64

In [7]:
    !pip install scikit-learn

     ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.9 MB 495.5 kB/s eta 0:00:18
     - -------------------------------------- 0.3/8.9 MB 2.5 MB/s eta 0:00:04
     ---- ----------------------------------- 1.1/8.9 MB 6.8 MB/s eta 0:00:02
     ------- -------------------------------- 1.6/8.9 MB 7.7 MB/s eta 0:00:01
     ---------- ----------------------------- 2.4/8.9 MB 9.7 MB/s eta 0:00:01
     ------------ --------------------------- 2.9/8.9 MB 10.2 MB/s eta 0:00:01
     ------------- -------------------------- 3.0/8.9 MB 8.4 MB/s eta 0:00:01
     ----------------- ---------------------- 3.9/8.9 MB 9.5 MB/s eta 0:00:01
     ------------------- -------------------- 4.4/8.9 MB 10.1 MB/s eta 0:00:01
     ----------------------- ---------------- 5.3/8.9 MB 10.9 MB/s eta 0:00:01
     -------------------------- ------------- 6.0/8.9 MB 11.2 MB/s eta 0:


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import os
import sys
sys.path.append(os.getcwd() + "/../src")

from cleaning import fill_missing_median, drop_missing, normalize_data

df = df_raw.copy()

df = fill_missing_median(df)

critical_cols = []
df = drop_missing(df, subset=critical_cols or None, how="any")
df_scaled = normalize_data(df, method="standard")

df_scaled.info()


<class 'pandas.core.frame.DataFrame'>
Index: 125 entries, 1 to 125
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    125 non-null    datetime64[ns]
 1   Close   125 non-null    object        
 2   High    125 non-null    object        
 3   Low     125 non-null    object        
 4   Open    125 non-null    object        
 5   Volume  125 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 6.8+ KB


In [19]:
def compare_before_after(df0: pd.DataFrame, df1: pd.DataFrame):
    comp = {}
    comp["rows_before"] = len(df0)
    comp["rows_after"]  = len(df1)
    comp["cols_before"] = df0.shape[1]
    comp["cols_after"]  = df1.shape[1]
    comp["na_before_total"] = int(df0.isna().sum().sum())
    comp["na_after_total"]  = int(df1.isna().sum().sum())
    num0 = df0.select_dtypes(include="object")
    num1 = df1.select_dtypes(include="object")
    comp["example_cols"] = list(num1.columns[:5])
    summ0 = num0.describe().round(3)
    summ1 = num1.describe().round(3)
    return comp, summ0, summ1

summary, before_desc, after_desc = compare_before_after(df_raw, df_scaled)
summary, before_desc, after_desc


({'rows_before': 126,
  'rows_after': 125,
  'cols_before': 6,
  'cols_after': 6,
  'na_before_total': 1,
  'na_after_total': 0,
  'example_cols': ['Close', 'High', 'Low', 'Open', 'Volume']},
                      Close High                 Low   Open Volume
 count                  126  126                 126    126    126
 unique                 125  126                 125    124    126
 top     243.44715881347656  JPM  296.20001220703125  291.5    JPM
 freq                     2    1                   2      2      1,
                      Close                High                 Low  \
 count                  125                 125                 125   
 unique                 124                 125                 124   
 top     243.44715881347656  266.26924263014877  296.20001220703125   
 freq                     2                   1                   2   
 
                      Open    Volume  
 count                 125       125  
 unique                123       125 

In [17]:
df_raw.dttypes

AttributeError: 'DataFrame' object has no attribute 'dttypes'